***Importação das bibliotecas***

In [1]:
import dask.dataframe as dd
import os
from dask.diagnostics import ProgressBar
from google.colab import drive

***Monta o Google Drive para acessar os arquivos de dados armazenados***

In [2]:
drive.mount('/content/drive')
directory_path = "/content/drive/MyDrive/Colab Notebooks/Base de Dados"
print("Arquivos no diretório:", os.listdir(directory_path))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Arquivos no diretório: ['COTAHIST_A2021.TXT', 'COTAHIST_A2022.TXT', 'COTAHIST_A2023.TXT', 'COTAHIST_A2024.TXT']


***Especifica as colunas e seus limites no arquivo de largura fixa***

In [3]:
colspecs = [
    (0, 2), (2, 10), (10, 12), (12, 24), (24, 27), (27, 39), (39, 49), (49, 52), (52, 56),
    (56, 69), (69, 82), (82, 95), (95, 108), (108, 121), (121, 134), (134, 147), (147, 152),
    (152, 170), (170, 188), (188, 201), (201, 202), (202, 210), (210, 217), (217, 230), (230, 242), (242, 245)
]
col_names = ["Tipo de Registro", "Data do pregão", "Código BDI", "Código de negociação", "Tipo de mercado", "Nome resumido",
             "Especificação do papel", "Prazo em dias", "Moeda de referência", "Preço de abertura",
             "Preço máximo", "Preço mínimo", "Preço médio", "Preço do último negócio", "Preço da melhor oferta de compra",
             "Preço da melhor oferta de venda", "Número de negócios", "Quantidade total de títulos", "Volume total",
             "Preço de exercício ou valor do contrato", "Indicador de correção", "Data de vencimento",
             "Fator de cotação", "Preço de exercício em pontos", "Código ISIN ou interno", "Número de distribuição"]

***Processa cada arquivo de dados históricos de cotações***

In [4]:
dfs = []
for filename in os.listdir(directory_path):
    if filename.startswith('COTAHIST_A'):
        file_path = os.path.join(directory_path, filename)
        df = dd.read_fwf(file_path, colspecs=colspecs, names=col_names, dtype=str, encoding='ISO-8859-1')
        filtered_df = df[df['Tipo de Registro'] == '01']  # Lazy filter operation
        dfs.append(filtered_df)

***Concatena todos os dataframes do Dask***

In [5]:
final_df = dd.concat(dfs, axis=0)

***Usando ProgressBar para visualizar o progresso da computação***

In [6]:
with ProgressBar():
    result = final_df.compute()  # Converte de Dask DataFrame para Pandas DataFrame somente no final

print("DataFrame final criado com sucesso. Total de linhas: ", result.shape[0])
print(result.head())

[########################################] | 100% Completed | 245.10 s
DataFrame final criado com sucesso. Total de linhas:  6894782
  Tipo de Registro Data do pregão Código BDI Código de negociação  \
1               01       20210104         02               A1AP34   
2               01       20210104         02               A1BB34   
3               01       20210104         02               A1BM34   
4               01       20210104         02               A1CR34   
5               01       20210104         02               A1EE34   

  Tipo de mercado Nome resumido Especificação do papel Prazo em dias  \
1             010  ADVANCE AUTO                    DRN          <NA>   
2             010       ABB LTD                    DRN          <NA>   
3             010   ABIOMED INC                    DRN          <NA>   
4             010     AMCOR PLC                    DRN          <NA>   
5             010   AMEREN CORP                    DRN          <NA>   

  Moeda de referênc

***Calcula e imprime a quantidade de valores nulos (NaN) por coluna***

In [7]:
# Supondo que 'final_df' é o seu DataFrame Dask final
# Calcular a quantidade de NaNs em cada coluna de forma lazy
na_counts = final_df.isna().sum()

# Usar ProgressBar para visualizar o progresso da computação
from dask.diagnostics import ProgressBar
with ProgressBar():
    na_result = na_counts.compute()

# Imprimir os resultados
print(na_result)

[########################################] | 100% Completed | 228.67 s
Tipo de Registro                                 0
Data do pregão                                   0
Código BDI                                       0
Código de negociação                             0
Tipo de mercado                                  0
Nome resumido                                    0
Especificação do papel                           0
Prazo em dias                              1337021
Moeda de referência                              0
Preço de abertura                                0
Preço máximo                                     0
Preço mínimo                                     0
Preço médio                                      0
Preço do último negócio                          0
Preço da melhor oferta de compra                 0
Preço da melhor oferta de venda                  0
Número de negócios                               0
Quantidade total de títulos                      0
Volume tota

***Imprimir os tipos de dados de cada coluna no DataFrame Dask***

In [8]:
print(final_df.dtypes)

Tipo de Registro                           string[pyarrow]
Data do pregão                             string[pyarrow]
Código BDI                                 string[pyarrow]
Código de negociação                       string[pyarrow]
Tipo de mercado                            string[pyarrow]
Nome resumido                              string[pyarrow]
Especificação do papel                     string[pyarrow]
Prazo em dias                              string[pyarrow]
Moeda de referência                        string[pyarrow]
Preço de abertura                          string[pyarrow]
Preço máximo                               string[pyarrow]
Preço mínimo                               string[pyarrow]
Preço médio                                string[pyarrow]
Preço do último negócio                    string[pyarrow]
Preço da melhor oferta de compra           string[pyarrow]
Preço da melhor oferta de venda            string[pyarrow]
Número de negócios                         string[pyarro

***Substituir os valores NaN da coluna Prazo em Dias por 0***

In [9]:
final_df['Prazo em dias'] = final_df['Prazo em dias'].fillna("0")

***Visualizar a quantidade de registros NaN após a substituição***

In [10]:
# Supondo que 'final_df' é o seu DataFrame Dask final
# Calcular a quantidade de NaNs em cada coluna de forma lazy
na_counts = final_df.isna().sum()

# Usar ProgressBar para visualizar o progresso da computação
from dask.diagnostics import ProgressBar
with ProgressBar():
    na_result = na_counts.compute()

# Imprimir os resultados
print(na_result)

[########################################] | 100% Completed | 236.43 s
Tipo de Registro                           0
Data do pregão                             0
Código BDI                                 0
Código de negociação                       0
Tipo de mercado                            0
Nome resumido                              0
Especificação do papel                     0
Prazo em dias                              0
Moeda de referência                        0
Preço de abertura                          0
Preço máximo                               0
Preço mínimo                               0
Preço médio                                0
Preço do último negócio                    0
Preço da melhor oferta de compra           0
Preço da melhor oferta de venda            0
Número de negócios                         0
Quantidade total de títulos                0
Volume total                               0
Preço de exercício ou valor do contrato    0
Indicador de correção        

***Alterando os tipos de registros das colunas***

In [11]:
# Função para converter colunas para int64, com fallback para -1 para valores não convertíveis
def safe_convert_to_int(df, column_name):
    # Converter a coluna para string primeiro, para garantir que o método .str possa ser aplicado
    temp_column = df[column_name].astype(str)
    # Aplicar a conversão para int, onde possível
    return dd.to_numeric(temp_column, errors='coerce').fillna(-1).astype('int64')

# Aplicar a função para converter "Tipo de Registro", "Tipo de mercado" e agora "Número de negócios"
final_df['Tipo de Registro'] = safe_convert_to_int(final_df, 'Tipo de Registro')
final_df['Tipo de mercado'] = safe_convert_to_int(final_df, 'Tipo de mercado')
final_df['Número de negócios'] = safe_convert_to_int(final_df, 'Número de negócios')  # Conversão para int64
final_df['Fator de cotação'] = safe_convert_to_int(final_df, 'Fator de cotação')  # Conversão para int64

# Converter "Data do pregão" para o tipo data
final_df['Data do pregão'] = dd.to_datetime(final_df['Data do pregão'], format='%Y%m%d', errors='coerce')

# Converter "Quantidade total de títulos" para inteiro sem casas decimais
final_df['Quantidade total de títulos'] = safe_convert_to_int(final_df, 'Quantidade total de títulos')

# Para colunas monetárias, converter para float, tratar NaNs e ajustar a escala dividindo por 100
monetary_columns = ['Preço de abertura', 'Preço máximo', 'Preço mínimo', 'Preço médio', 'Preço do último negócio',
                    'Preço da melhor oferta de compra', 'Preço da melhor oferta de venda', 'Volume total',
                    'Preço de exercício ou valor do contrato']
for col in monetary_columns:
    final_df[col] = final_df[col].astype(float).fillna(0) / 100

# Computar somente se e quando necessário, de preferência em partes
result = final_df.head()  # Executa de forma lazy apenas para visualização inicial
print(result)




   Tipo de Registro Data do pregão Código BDI Código de negociação  \
1                 1     2021-01-04         02               A1AP34   
2                 1     2021-01-04         02               A1BB34   
3                 1     2021-01-04         02               A1BM34   
4                 1     2021-01-04         02               A1CR34   
5                 1     2021-01-04         02               A1EE34   

   Tipo de mercado Nome resumido Especificação do papel Prazo em dias  \
1               10  ADVANCE AUTO                    DRN             0   
2               10       ABB LTD                    DRN             0   
3               10   ABIOMED INC                    DRN             0   
4               10     AMCOR PLC                    DRN             0   
5               10   AMEREN CORP                    DRN             0   

  Moeda de referência  Preço de abertura  ...  Número de negócios  \
1                  R$              51.10  ...                  22   
2 

***Salvando o DataFrame em um arquivo CSV para ser utilizado no modelo***

In [12]:
# Define o caminho para salvar o arquivo CSV
output_csv_path = os.path.join(directory_path, "final_data_for_ml.csv")

# Usar ProgressBar para visualizar o progresso da computação
with ProgressBar():
    # Salvar o DataFrame como CSV
    final_df.to_csv(output_csv_path, single_file=True, index=False)

print("DataFrame foi salvo com sucesso no formato CSV em:", output_csv_path)

# Amostrar 10% do DataFrame para o conjunto de testes
test_df = final_df.sample(frac=0.1, random_state=42)

# Define o caminho para salvar o arquivo CSV do conjunto de testes
test_csv_path = os.path.join(directory_path, "test_data_for_ml.csv")

# Usar ProgressBar para visualizar o progresso da computação
with ProgressBar():
    # Salvar o conjunto de testes como CSV
    test_df.to_csv(test_csv_path, single_file=True, index=False)

print("Conjunto de dados de teste foi salvo em:", test_csv_path)


[########################################] | 100% Completed | 417.04 s
DataFrame foi salvo com sucesso no formato CSV em: /content/drive/MyDrive/Colab Notebooks/Base de Dados/final_data_for_ml.csv
[########################################] | 100% Completed | 306.98 s
Conjunto de dados de teste foi salvo em: /content/drive/MyDrive/Colab Notebooks/Base de Dados/test_data_for_ml.csv
